In [ ]:
# Trying to understand intent and extract entities

import re

keywords = {
 'goodbye': ['bye', 'farewell'],
 'greet': ['hello', 'hi', 'hey'],
 'thankyou': ['thank', 'thx']
}

responses = {
 'goodbye': ['bye', 'farewell'],
 'greet': ['hello', 'hi', 'hey'],
 'thankyou': ['thank', 'thx']
}

# Define a dictionary of patterns
patterns = {}

# Iterate over the keywords dictionary
for intent, keys in keywords.items():
    # Create regular expressions and compile them into pattern objects
    patterns[intent] = re.compile('|'.join(keys))
    
# Print the patterns
print(patterns)

In [ ]:
import random

bot_template = "BOT : {0}"
user_template = "USER : {0}"

# Define a function that sends a message to the bot: send_message
def send_message(message):
    # Print user_template including the user_message
    print(user_template.format(message))
    # Get the bot's response to the message
    response = respond(message)
    # Print the bot template including the bot's response.
    print(bot_template.format(response))

# Define a function to find the intent of a message
def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message):
            matched_intent = intent
    return matched_intent

# Define a respond function
def respond(message):
    # Call the match_intent function
    intent = match_intent(message)
    # Fall back to the default response
    key = "default"
    if intent in responses:
        key = intent
    return random.choice(responses[key])

# Send messages
send_message("hello!")
send_message("bye byeee")
send_message("thanks very much!")

In [ ]:
# Define find_name()
def find_name(message):
    name = None
    # Create a pattern for checking if the keywords occur
    name_keyword = re.compile(r"(name|call)")
    # Create a pattern for finding capitalized words
    name_pattern = re.compile(r"(\b[A-Z][a-z]*\b)")
    if name_keyword.search(message):
        # Get the matching words in the string
        name_words = name_pattern.findall(message)
        if len(name_words) > 0:
            # Return the name if the keywords are present
            name = ' '.join(name_words)
    return name

# Define respond()
def respond(message):
    # Find the name
    name = find_name(message)
    if name is None:
        return "Hi there!"
    else:
        return "Hello, {0}!".format(name)

# Send messages
send_message("my name is David Copperfield")
send_message("call me Ishmael")
send_message("People call me Cassandra")


In [ ]:
import csv
with open('atis_intents.csv', 'r') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    labels = []
    sentences = []
    for row in readCSV:
        label = row[0]
        sentence = row[1]
        labels.append(label)
        sentences.append(sentence)


In [ ]:
!python3 -m spacy download en_core_web_sm

In [ ]:
print(sentences[0])
print(labels[0])

In [ ]:
!python3 -m spacy download en_vectors_web_lg


In [ ]:
import spacy
import numpy as np

# Load the spacy model: nlp
nlp = spacy.load('en_vectors_web_lg')

# Calculate the length of sentences
n_sentences = len(sentences)

print(n_sentences)

# Calculate the dimensionality of nlp
embedding_dim = nlp.vocab.vectors_length

print(embedding_dim)

# Initialize the array with zeros: X
X = np.zeros((n_sentences, embedding_dim))

# Iterate over the sentences
for idx, sentence in enumerate(sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    # Save the document's .vector attribute to the corresponding row in X
    X[idx, :] = doc.vector

In [ ]:
# Intent classification with sklearn

# Import SVC
from sklearn.svm import SVC
# X_train and y_train was given.

# Create a support vector classifier
clf = SVC(C=1)

# Fit the classifier using the training data
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Count the number of correct predictions
n_correct = 0
for i in range(len(y_test)):
    if y_pred[i] == y_test[i]:
        n_correct += 1

print("Predicted {0} correctly out of {1} test examples".format(n_correct, len(y_test)))


In [ ]:
# Define included entities
include_entities = ['DATE', 'ORG', 'PERSON']

# Define extract_entities()
def extract_entities(message):
    # Create a dict to hold the entities
    ents = dict.fromkeys(include_entities)
    # Create a spacy document
    doc = nlp(message)
    for ent in doc.ents:
        if ent.label_ in include_entities:
            # Save interesting entities
            ents[ent.label_] = ent.text
    return ents

print(extract_entities('friends called Mary who have worked at Google since 2010'))
print(extract_entities('people who graduated from MIT in 1999'))


In [ ]:
# Create the document
doc = nlp("let's see that jacket in red and some blue jeans")

# Iterate over parents in parse tree until an item entity is found
def find_parent_item(word):
    # Iterate over the word's ancestors
    for parent in word.ancestors:
        # Check for an "item" entity
        if entity_type(parent) == "item":
            return parent.text
    return None

# For all color entities, find their parent item
def assign_colors(doc):
    # Iterate over the document
    for word in doc:
        # Check for "color" entities
        if entity_type(word) == "color":
            # Find the parent
            item = find_parent_item(word)
            print("item: {0} has color : {1}".format(item, word))

# Assign the colors
assign_colors(doc)


In [ ]:
# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer

# Create args dictionary
args = {"pipeline": "spacy_sklearn"}

# Create a configuration and trainer
config = RasaNLUModelConfig(args)
print(config)
trainer = Trainer(config)

# Load the training data
training_data = load_data("./sss.json")

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

# Try it out
print(interpreter.parse("I'm looking for a Mexican restaurant in the North of town"))


In [ ]:
print(interpreter.parse("tell me your name"))

In [ ]:
!pip install tensorflow==1.14

In [1]:
# Data-efficient entity recognition

# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config
from rasa_nlu.model import Metadata, Interpreter
# from tensorflow import set_random_seed
# tf.set_random_seed(42) 
# sess = tf.Session()
args = {"pipeline": 
    "tensorflow_embedding"}

# Create a config that uses this pipeline 
config = RasaNLUModelConfig(args)
# Create a trainer that uses this config
trainer = Trainer(config)
training_data = load_data("sss.json")
# Create an interpreter by training the model
interpreter = trainer.train(training_data)
model_dir=trainer.persist('project/',fixed_model_name = 'test')
interpreter=interpreter.load(model_dir)

# Parse some messages
#print(interpreter.parse("show me Chinese food in the centre of town"))
#print(interpreter.parse("I want an Indian restaurant in the west"))
#print(interpreter.parse("are there any good pizza places in the center?"))


/home/mitho/DemoChatBot2/DemoChatBot/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mitho/DemoChatBot2/DemoChatBot/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mitho/DemoChatBot2/DemoChatBot/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mitho/DemoChatBot2/Dem




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dropout instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Epochs: 100%|██████████| 1900/1900 [01:53<00:00, 16.73it/s, loss=0.165, acc=1.000]


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/mitho/DemoChatBot2/DemoChatBot/project/default/test/component_5_EmbeddingIntentClassifier.ckpt


In [ ]:
from rasa_nlu.model import Metadata, Interpreter
interpreter = Interpreter.load('/home/mitho/DemoChatBot2/DemoChatBot/project/default/test/')

In [3]:
import pandas as pd
df=pd.read_csv("file3.csv",encoding='mac_roman')
df=df.set_index('Intents')

In [4]:
df

,Answers
Intents,
ask_sup,"Oh, Just the usual!"
ask_health,"Very well, thanks."
Ask_gender,"Yes, I am a pretty girl."
Ask_born_place,I was born in Japan.
ask_religion,I am proud that I am an American citizen of Ja...
...,...
ask_acne,Unfortunately Yes! whenever I used to eat exce...
Ask_arms,"Yes, I have 2 forelimbs."
Ask_eyes,"Yes, I have very beautiful greenish eyes."


In [5]:
username = input("Enter username:")
a=interpreter.parse(username)
print(a)
b=a['intent']['name']
c = a['intent']['confidence']
t=a['entities']
print(t)
print(c)
print(b)
if c>=0.80:
    print(df.loc[b].values[0])
else:
    print("doesn't get your point can you ask") 

Enter username:favorite book
{'intent': {'name': 'ask_book', 'confidence': 0.9284505248069763}, 'entities': [], 'intent_ranking': [{'name': 'ask_book', 'confidence': 0.9284505248069763}, {'name': 'ask_yawn', 'confidence': 0.3006148338317871}, {'name': 'ask_eyes', 'confidence': 0.2680564522743225}, {'name': 'ask_lead', 'confidence': 0.2469063401222229}, {'name': 'ask_tired', 'confidence': 0.2287183403968811}, {'name': 'ask_date_birth', 'confidence': 0.21841317415237427}, {'name': 'ask_health', 'confidence': 0.21036700904369354}, {'name': 'ask_country', 'confidence': 0.20145606994628906}, {'name': 'ask_life_stage', 'confidence': 0.19621795415878296}, {'name': 'ask_glasses', 'confidence': 0.19499269127845764}], 'text': 'favorite book'}
[]
0.9284505248069763
ask_book
The Millionaire Next Door is my favorite book
